<a href="https://colab.research.google.com/github/aljeshishe/notebooks/blob/master/How_much_samples_is_enough_for_transfer_learning_same_steps_per_epoch_sweep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install kaggle -q

In [0]:
import json
token = {'username':'aljeshishe','key':'32deca82aa1c29fbaeadcce2bf470af4'}
with open('kaggle.json', 'w') as file:
    json.dump(token, file)

In [28]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [29]:
!kaggle competitions download -c dogs-vs-cats

sampleSubmission.csv: Skipping, found more recently modified local copy (use --force to force download)
test1.zip: Skipping, found more recently modified local copy (use --force to force download)
train.zip: Skipping, found more recently modified local copy (use --force to force download)


In [30]:
!unzip test1.zip > /dev/null
!unzip train.zip > /dev/null

replace test1/1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace train/cat.0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [2]:
!nvidia-smi

Running on single GPU  /physical_device:GPU:0
Number of accelerators:  1
Mon May 11 08:12:08 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    28W / 250W |     10MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:   

In [8]:
from google.colab import drive
drive.mount('/content/drive')

def notebook_name():
  import re
  import ipykernel
  import requests

  from notebook.notebookapp import list_running_servers
  # kernel_id = re.search('kernel-(.*).json', ipykernel.connect.get_connection_file()).group(1)
  for ss in list_running_servers():
      response = requests.get(f'{ss["url"]}api/sessions',params={'token': ss.get('token', '')})
      return response.json()[0]['name']


project, _, _ = notebook_name().rpartition('.')

import re 
project = re.sub('[^-a-zA-Z0-9_]+', '_', project)

working_dir = f'/content/drive/My Drive/Colab Notebooks/{project}'
print(f'Current project: {project}')
print(f'Places at: {working_dir}')

import pathlib
pathlib.Path(working_dir).mkdir(parents=True, exist_ok=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current project: Copy_20of_20How_much_samples_is_enough_for_transfer_learning_same_steps_per_epoch_sweep
Places at: /content/drive/My Drive/Colab Notebooks/Copy_20of_20How_much_samples_is_enough_for_transfer_learning_same_steps_per_epoch_sweep


In [0]:
!pip install wandb -q

In [0]:
pip install image-classifiers -q

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random

import os

train_dir = 'train/'
test_dir = 'test1/'
filenames = os.listdir(train_dir)
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append("1")
    else:
        categories.append("0")

In [0]:
from keras.models import Sequential
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.applications import VGG16
from keras.applications import InceptionResNetV2
from keras.models import Model
from keras.callbacks import EarlyStopping
import wandb
from wandb.keras import WandbCallback
from classification_models.keras import Classifiers



def run():
    config = dict(
      batch_size=64,
      epochs=10,
      lr=1e-4,
      momentum=0.9,
      steps_per_epoch=20000/64,
      model_name="vgg16",
      samples=10)

    run = wandb.init(dir=working_dir, config=config)
    conf = run.config
    print(conf)

    df = pd.DataFrame({
        'filename': filenames,
        'category': categories
    })
    df.head()

    image_size = 224
    input_shape = (image_size, image_size, 3)

    epochs = conf.epochs
    batch_size = conf.batch_size

    ModelClass, preprocess_input = Classifiers.get(conf.model_name)
    pre_trained_model = ModelClass(input_shape=input_shape, include_top=False, weights="imagenet")
    # pre_trained_model.summary()

    for layer in pre_trained_model.layers:
        layer.trainable = False

    # for layer in pre_trained_model.layers:
    #     layer.trainable = True

    last_layer = pre_trained_model.layers[-1]
    last_output = last_layer.output

    x = GlobalMaxPooling2D()(last_output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = layers.Dense(1, activation='sigmoid')(x)

    model = Model(pre_trained_model.input, x)

    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.SGD(lr=conf.lr, momentum=conf.momentum),
                  metrics=['accuracy'])

    train_df, validate_df = train_test_split(df, test_size=0.1)
    train_df = train_df.reset_index()
    validate_df = validate_df.reset_index()

    validate_df = validate_df.sample(n=2000).reset_index() # use for fast testing code purpose
    train_df = train_df.sample(n=conf.samples).reset_index() # use for fast testing code purpose

    total_train = train_df.shape[0]
    total_validate = validate_df.shape[0]

    train_datagen = ImageDataGenerator(
        rotation_range=15,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        width_shift_range=0.1,
        height_shift_range=0.1
    )

    train_generator = train_datagen.flow_from_dataframe(
        train_df, 
        train_dir, 
        x_col='filename',
        y_col='category',
        class_mode='binary',
        target_size=(image_size, image_size),
        batch_size=batch_size
    )

    validation_datagen = ImageDataGenerator(rescale=1./255)
    validation_generator = validation_datagen.flow_from_dataframe(
        validate_df, 
        train_dir, 
        x_col='filename',
        y_col='category',
        class_mode='binary',
        target_size=(image_size, image_size),
        batch_size=batch_size
    )

    # fine-tune the model
    history = model.fit_generator(
        train_generator,
        epochs=conf.epochs,
        validation_data=validation_generator,
        validation_steps=total_validate//conf.batch_size,
        steps_per_epoch=conf.steps_per_epoch,
        verbose=1,
        callbacks=[WandbCallback(save_model=True, verbose=1)])
        #keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
    return history


In [0]:

!WANDB_API_KEY=723983b2d42ccd7c5510bbeb0549aa73f1242844
!export WANDB_API_KEY

import wandb
sweep_config = {
  "name": "My Sweep",
  "method": "grid",
  "parameters": {
        "samples": {
            "values": [10, 50, 100, 1000, 20000]
        },
        "model_name": {
            "values": ["vgg16", "resnext101", "inceptionresnetv2", "nasnetlarge"]
        }

    }
}


sweep_id = wandb.sweep(sweep_config, project=project)
wandb.agent(sweep_id, function=run)

Create sweep with ID: djxmo76x
Sweep URL: https://app.wandb.ai/grachev/Copy_20of_20How_much_samples_is_enough_for_transfer_learning_same_steps_per_epoch_sweep/sweeps/djxmo76x
wandb: Agent Starting Run: vrfqg2i0 with config:
	model_name: vgg16
	samples: 10
wandb: Agent Started Run: vrfqg2i0


wandb_version: 1

_wandb:
  desc: null
  value:
    cli_version: 0.8.35
    framework: keras
    is_jupyter_run: true
    is_kaggle_kernel: false
    python_version: 3.6.9
batch_size:
  desc: null
  value: 64
epochs:
  desc: null
  value: 10
lr:
  desc: null
  value: 0.0001
model_name:
  desc: null
  value: vgg16
momentum:
  desc: null
  value: 0.9
samples:
  desc: null
  value: 10
steps_per_epoch:
  desc: null
  value: 312.5

Found 10 validated image filenames belonging to 2 classes.
Found 2000 validated image filenames belonging to 2 classes.
Epoch 1/10
313/312 [==============================] - 72s 232ms/step - loss: 0.3952 - accuracy: 0.8313 - val_loss: 0.5397 - val_accuracy: 0.6048
Epoch 00000: val_loss improved from inf to 0.53965, saving model to /content/wandb/run-20200511_084419-vrfqg2i0/model-best.h5
Epoch 2/10
313/312 [==============================] - 70s 225ms/step - loss: 0.1728 - accuracy: 0.9645 - val_loss: 0.5585 - val_accuracy: 0.6420
Epoch 3/10
313/312 [=============

wandb_version: 1

_wandb:
  desc: null
  value:
    cli_version: 0.8.35
    framework: keras
    is_jupyter_run: true
    is_kaggle_kernel: false
    python_version: 3.6.9
batch_size:
  desc: null
  value: 64
epochs:
  desc: null
  value: 10
lr:
  desc: null
  value: 0.0001
model_name:
  desc: null
  value: vgg16
momentum:
  desc: null
  value: 0.9
samples:
  desc: null
  value: 50
steps_per_epoch:
  desc: null
  value: 312.5

Found 50 validated image filenames belonging to 2 classes.
Found 2000 validated image filenames belonging to 2 classes.
Epoch 1/10
313/312 [==============================] - 237s 758ms/step - loss: 0.6599 - accuracy: 0.6298 - val_loss: 0.6455 - val_accuracy: 0.7213
Epoch 00000: val_loss improved from inf to 0.64547, saving model to /content/wandb/run-20200511_085614-pmxcobwm/model-best.h5
Epoch 2/10
270/312 [========================>.....] - ETA: 30s - loss: 0.5055 - accuracy: 0.7581

wandb: Network error resolved after 0:00:18.927409, resuming normal operation.


282/312 [==========================>...] - ETA: 22s - loss: 0.5035 - accuracy: 0.7598

...

wandb_version: 1

_wandb:
  desc: null
  value:
    cli_version: 0.8.35
    framework: keras
    is_jupyter_run: true
    is_kaggle_kernel: false
    python_version: 3.6.9
batch_size:
  desc: null
  value: 64
epochs:
  desc: null
  value: 10
lr:
  desc: null
  value: 0.0001
model_name:
  desc: null
  value: vgg16
momentum:
  desc: null
  value: 0.9
samples:
  desc: null
  value: 20000
steps_per_epoch:
  desc: null
  value: 312.5

Found 20000 validated image filenames belonging to 2 classes.
Found 2000 validated image filenames belonging to 2 classes.
Epoch 1/10
313/312 [==============================] - 262s 836ms/step - loss: 0.7059 - accuracy: 0.5814 - val_loss: 0.4602 - val_accuracy: 0.8206
Epoch 00000: val_loss improved from inf to 0.46016, saving model to /content/wandb/run-20200511_105707-j9fp77sy/model-best.h5
Epoch 2/10
313/312 [==============================] - 253s 808ms/step - loss: 0.5485 - accuracy: 0.7161 - val_loss: 0.4267 - val_accuracy: 0.8528
Epoch 00001: val_loss imp

wandb_version: 1

_wandb:
  desc: null
  value:
    cli_version: 0.8.35
    framework: keras
    is_jupyter_run: true
    is_kaggle_kernel: false
    python_version: 3.6.9
batch_size:
  desc: null
  value: 64
epochs:
  desc: null
  value: 10
lr:
  desc: null
  value: 0.0001
model_name:
  desc: null
  value: resnext101
momentum:
  desc: null
  value: 0.9
samples:
  desc: null
  value: 10
steps_per_epoch:
  desc: null
  value: 312.5

Found 10 validated image filenames belonging to 2 classes.
Found 2000 validated image filenames belonging to 2 classes.
Epoch 1/10
  2/312 [..............................] - ETA: 29:52 - loss: 1.5356 - accuracy: 0.5000

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.223460). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


313/312 [==============================] - 98s 312ms/step - loss: 0.1179 - accuracy: 0.9601 - val_loss: 0.7904 - val_accuracy: 0.5101
Epoch 00000: val_loss improved from inf to 0.79039, saving model to /content/wandb/run-20200511_114010-n0gal6xf/model-best.h5
Epoch 2/10
313/312 [==============================] - 81s 258ms/step - loss: 0.0108 - accuracy: 0.9987 - val_loss: 0.7244 - val_accuracy: 0.5072
Epoch 00001: val_loss improved from 0.79039 to 0.72436, saving model to /content/wandb/run-20200511_114010-n0gal6xf/model-best.h5
Epoch 3/10
313/312 [==============================] - 79s 253ms/step - loss: 0.0058 - accuracy: 0.9987 - val_loss: 0.7587 - val_accuracy: 0.5093
Epoch 4/10
313/312 [==============================] - 79s 253ms/step - loss: 0.0039 - accuracy: 0.9994 - val_loss: 0.7657 - val_accuracy: 0.5088
Epoch 5/10
313/312 [==============================] - 79s 253ms/step - loss: 0.0033 - accuracy: 0.9997 - val_loss: 0.8497 - val_accuracy: 0.5129
Epoch 6/10
313/312 [==========

wandb_version: 1

_wandb:
  desc: null
  value:
    cli_version: 0.8.35
    framework: keras
    is_jupyter_run: true
    is_kaggle_kernel: false
    python_version: 3.6.9
batch_size:
  desc: null
  value: 64
epochs:
  desc: null
  value: 10
lr:
  desc: null
  value: 0.0001
model_name:
  desc: null
  value: resnext101
momentum:
  desc: null
  value: 0.9
samples:
  desc: null
  value: 50
steps_per_epoch:
  desc: null
  value: 312.5

Found 50 validated image filenames belonging to 2 classes.
Found 2000 validated image filenames belonging to 2 classes.
Epoch 1/10
313/312 [==============================] - 283s 905ms/step - loss: 0.1956 - accuracy: 0.9236 - val_loss: 1.0163 - val_accuracy: 0.5101
Epoch 00000: val_loss improved from inf to 1.01634, saving model to /content/wandb/run-20200511_115430-9kmu4tfz/model-best.h5
Epoch 2/10
313/312 [==============================] - 266s 850ms/step - loss: 0.0321 - accuracy: 0.9914 - val_loss: 1.0816 - val_accuracy: 0.5088
Epoch 3/10
313/312 [======

In [0]:
val_acc = [(i, max(result['val_accuracy'])) for i, result in zip(results_count, results)]
acc = [(i, max(result['accuracy'])) for i, result in zip(results_count, results)]
val_loss = [(i, min(result['val_loss'])) for i, result in zip(results_count, results)]
loss = [(i, min(result['loss'])) for i, result in zip(results_count, results)]
import matplotlib.pyplot as plt
plt.plot(*zip(*val_acc), '-o', label='val_acc')
plt.plot(*zip(*acc), '-o', label='acc')
plt.plot(*zip(*val_loss), '-o', label='val_loss')
plt.plot(*zip(*loss), '-o', label='loss')
plt.legend()

In [0]:
val_acc

https://app.wandb.ai/grachev/Copy_20of_20How_much_samples_is_enough_for_transfer_learning_same_steps_per_epoch_sweep/sweeps/djxmo76x/table?workspace=user-grachev

**500 samples is ok for binary classification**